In [1]:
import tensorflow as tf
print(tf.__version__)

2.4.1


In [2]:
print(tf.keras.__version__)

2.4.0


In [ ]:
!pip install tensorflow==2.4.1

In [3]:
%matplotlib inline
import PIL
import os
import tqdm
import pickle
import numpy as np
import matplotlib.pyplot as plt
from skimage.filters import gabor
from skimage.feature import local_binary_pattern, greycomatrix, greycoprops
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from skimage.io import imread, imshow
from PIL import Image
from skimage import data
from skimage.util import img_as_ubyte
from skimage.filters.rank import entropy
from skimage.morphology import disk
from skimage.color import rgb2hsv, rgb2gray, rgb2yuv
from skimage.filters.rank import entropy
from skimage.morphology import disk
import skimage.measure  
from fer import FER

In [4]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

# from tensorflow.keras.applications.vgg16 import VGG16
# from tensorflow.keras.preprocessing import image
# from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions

# from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
# from tensorflow.keras.preprocessing import image
# from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input, decode_predictions

import seaborn as sns
import requests
import urllib
from tensorflow.keras import backend as K
from tensorflow import keras 

In [5]:
def get_rgb_values(folder_path=None):
    image_bgr = cv2.imread(os.path.join(folder_path,filename), cv2.IMREAD_COLOR)
    channels = cv2.mean(image_bgr)
    observation = np.array([(channels[2], channels[1], channels[0])])
    return filename, channels[2], channels[1], channels[0]

In [6]:
def get_hsv_values(folder_path=None):
    img = cv2.imread(os.path.join(folder_path,filename))
    hsv = cv2.cvtColor (img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split (hsv)
    mean1 = h.mean ()
    mean2 = s.mean ()
    mean3 = v.mean ()
    return filename, mean1, mean2, mean3

In [7]:
def get_other_entropies(folder_path=None):
    image=cv2.imread(os.path.join(folder_path,filename))
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    shanon_entropy = skimage.measure.shannon_entropy(gray_image)
    return shanon_entropy

In [8]:
def get_filter_features(folder_path=None):
    with Image.open(os.path.join(folder_path,filename)) as im:
        img_gray = im.convert('L') #Converting to grayscale
        img_arr = np.array(img_gray) #Converting to array
        # Finding LBP
        feat_lbp = local_binary_pattern(img_arr,8,1,'uniform') #Radius = 1, No. of neighbours = 8
        feat_lbp = np.uint8((feat_lbp/feat_lbp.max())*255) #Converting to unit8
        lbp_img = PIL.Image.fromarray(feat_lbp) #Conversion from array to PIL image
        # Energy and Entropy of LBP feature
        lbp_hist,_ = np.histogram(feat_lbp,8)
        lbp_hist = np.array(lbp_hist,dtype=float)
        lbp_prob = np.divide(lbp_hist,np.sum(lbp_hist))
        lbp_energy = np.sum(lbp_prob**2)
        lbp_entropy = -np.sum(np.multiply(lbp_prob,np.log2(lbp_prob)))
        # Finding GLCM features from co-occurance matrix
        gCoMat = greycomatrix(img_arr, [2], [0],256,symmetric=True, normed=True) # Co-occurance matrix
        contrast = greycoprops(gCoMat, prop='contrast')
        dissimilarity = greycoprops(gCoMat, prop='dissimilarity')
        homogeneity = greycoprops(gCoMat, prop='homogeneity')
        energy = greycoprops(gCoMat, prop='energy')
        correlation = greycoprops(gCoMat, prop='correlation')
        contrast_new=contrast[0][0]
        dissimilarity_new=dissimilarity[0][0]
        homogeneity_new=homogeneity[0][0]
        energy_new=energy[0][0]
        correlation_new=correlation[0][0]
        # Gabor filter
        gaborFilt_real,gaborFilt_imag = gabor(img_arr,frequency=0.6)
        gaborFilt = (gaborFilt_real**2+gaborFilt_imag**2)//2
        # Energy and Entropy of Gabor filter response
        gabor_hist,_ = np.histogram(gaborFilt,8)
        gabor_hist = np.array(gabor_hist,dtype=float)
        gabor_prob = np.divide(gabor_hist,np.sum(gabor_hist))
        gabor_energy = np.sum(gabor_prob**2)
        gabor_entropy = -np.sum(np.multiply(gabor_prob,np.log2(gabor_prob))) 
        return lbp_energy,lbp_entropy, contrast_new, dissimilarity_new, homogeneity_new, energy_new, correlation_new,gabor_energy, gabor_entropy

In [9]:
def get_cognitive_demand(folder_path=None):
    Cognitive_Load=0
    image_read = cv2.imread(os.path.join(folder_path,filename))
    img_gray = cv2.cvtColor(image_read, cv2.COLOR_BGR2GRAY)
    entr_img = entropy(img_gray, disk(10))
    Local_Entropy=np.average(entr_img)  ### Here the average on taken on the whole image by weighting every pixel equally
    Cognitive_Load = (Local_Entropy/8)*100 ### Scaling it to 100
    return Cognitive_Load

In [10]:
def detect_face_emotion(folder_path=None):
    test_image_one = plt.imread(os.path.join(folder_path,filename))
    emo_detector = FER(mtcnn=True)
    dominant_emotion, emotion_score = emo_detector.top_emotion(test_image_one)
    return dominant_emotion, emotion_score

In [12]:
model_resnet = ResNet50(weights='imagenet')

def image_classification(folder_path=None):
    top_3_predictions=[]
    img = cv2.imread(os.path.join(folder_path,filename))
    img = cv2.resize(img, (224, 224))
    img = np.expand_dims(img, axis=0)
    img=  preprocess_input(img)
    preds_1 = model_resnet.predict(img)
    top_3_predictions.append(decode_predictions(preds_1, top=3)[0])
    return top_3_predictions

In [13]:
name_1_list=[]
name_2_list=[]
R_list=[]
G_list=[]
B_list=[]
H_list=[]
S_list=[]
V_list=[]
lbp_energy_list=[]
lbp_entropy_list=[]
contrast_new_list=[] 
dissimilarity_new_list=[]
homogeneity_new_list=[]
energy_new_list=[]
correlation_new_list=[]
gabor_energy_list=[]
gabor_entropy_list=[]
cognitive_demand_list=[]
shanon_entropy_list=[]
local_entropy_list=[]

In [14]:
folder_path="IESN_resnet_images"
results = pd.DataFrame()
count=0
for filename in os.listdir(folder_path):
    name_1,R,G,B=get_rgb_values(folder_path)
    name_2,H,S,V=get_hsv_values(folder_path)
    lbp_energy,lbp_entropy, contrast_new, dissimilarity_new, homogeneity_new, energy_new, correlation_new,gabor_energy, gabor_entropy = get_filter_features(folder_path)
    cognitive_demand=get_cognitive_demand(folder_path)
    shanon_entropy=get_other_entropies(folder_path)
    name_1_list.append(name_1)
    name_2_list.append(name_2)
    R_list.append(R)
    G_list.append(G)
    B_list.append(B)
    H_list.append(H)
    S_list.append(S)
    V_list.append(V)
    lbp_energy_list.append(lbp_energy)
    lbp_entropy_list.append(lbp_entropy)
    contrast_new_list.append(contrast_new)
    dissimilarity_new_list.append(dissimilarity_new)
    homogeneity_new_list.append(homogeneity_new)
    energy_new_list.append(energy_new)
    correlation_new_list.append(correlation_new)
    gabor_energy_list.append(gabor_energy)
    gabor_entropy_list.append(gabor_entropy)
    cognitive_demand_list.append(cognitive_demand)
    shanon_entropy_list.append(shanon_entropy)
    count=count+1
    print("Done, Processed Image {}, Total Images processed: {}".format(name_1,count))

Done, Processed Image I000683589.jpg, Total Images processed: 1
Done, Processed Image I000241412.jpg, Total Images processed: 2
Done, Processed Image I000597539.jpg, Total Images processed: 3
Done, Processed Image I000399706.jpg, Total Images processed: 4
Done, Processed Image I000308594.jpg, Total Images processed: 5
Done, Processed Image I000391755.jpg, Total Images processed: 6
Done, Processed Image I000739145.jpg, Total Images processed: 7
Done, Processed Image I000342349.jpg, Total Images processed: 8
Done, Processed Image I000256886.jpg, Total Images processed: 9
Done, Processed Image I000493817.jpg, Total Images processed: 10
Done, Processed Image I000500919.jpg, Total Images processed: 11
Done, Processed Image I000294417.jpg, Total Images processed: 12
Done, Processed Image I000270821.jpg, Total Images processed: 13
Done, Processed Image I000754861.jpg, Total Images processed: 14
Done, Processed Image I000333940.jpg, Total Images processed: 15
Done, Processed Image I000115060.j

/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/ipykernel/__main__.py:35: RuntimeWarning: divide by zero encountered in log2
/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/ipykernel/__main__.py:35: RuntimeWarning: invalid value encountered in multiply


Done, Processed Image I000393337.jpg, Total Images processed: 190
Done, Processed Image I000249542.jpg, Total Images processed: 191
Done, Processed Image I000736831.jpg, Total Images processed: 192
Done, Processed Image I000181620.jpg, Total Images processed: 193
Done, Processed Image I000084056.jpg, Total Images processed: 194
Done, Processed Image I000256981.jpg, Total Images processed: 195
Done, Processed Image I000434838.jpg, Total Images processed: 196
Done, Processed Image I000673335.jpg, Total Images processed: 197
Done, Processed Image I000659087.jpg, Total Images processed: 198
Done, Processed Image I000939900.jpg, Total Images processed: 199
Done, Processed Image I000363251.jpg, Total Images processed: 200
Done, Processed Image I000106535.jpg, Total Images processed: 201
Done, Processed Image I000425872.jpg, Total Images processed: 202
Done, Processed Image I000937433.jpg, Total Images processed: 203
Done, Processed Image I000739617.jpg, Total Images processed: 204
Done, Proc

/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/ipykernel/__main__.py:14: RuntimeWarning: divide by zero encountered in log2
/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/ipykernel/__main__.py:14: RuntimeWarning: invalid value encountered in multiply


Done, Processed Image I000348775.jpg, Total Images processed: 16572
Done, Processed Image I000804155.jpg, Total Images processed: 16573
Done, Processed Image I000071405.jpg, Total Images processed: 16574
Done, Processed Image I000588587.jpg, Total Images processed: 16575
Done, Processed Image I000826524.jpg, Total Images processed: 16576
Done, Processed Image I000368373.jpg, Total Images processed: 16577
Done, Processed Image I000811183.jpg, Total Images processed: 16578
Done, Processed Image I000649793.jpg, Total Images processed: 16579
Done, Processed Image I000507184.jpg, Total Images processed: 16580
Done, Processed Image I000739897.jpg, Total Images processed: 16581
Done, Processed Image I000579953.jpg, Total Images processed: 16582
Done, Processed Image I000687774.jpg, Total Images processed: 16583
Done, Processed Image I000447767.jpg, Total Images processed: 16584
Done, Processed Image I000981118.jpg, Total Images processed: 16585
Done, Processed Image I000295700.jpg, Total Imag

In [ ]:
# shanon_entropy_list

In [15]:
results = pd.DataFrame(
    {'name_1': name_1_list,
     'name_2': name_2_list,
     'R': R_list,
     'G': G_list,
     'B': B_list,
     'H': H_list,
     'S': S_list,
     'V': V_list,
    'lbp_energy': lbp_energy_list,
    'lbp_entropy': lbp_entropy_list,
    'contrast_new': contrast_new_list,
    'dissimilarity_new': dissimilarity_new_list,
    'homogeneity_new': homogeneity_new_list,
    'energy_new': energy_new_list,
    'correlation_new': correlation_new_list,
    'gabor_energy':gabor_energy_list,
    'gabor_entropy': gabor_entropy_list,
    'cognitive_demand': cognitive_demand_list,
    'shanon_entropy': shanon_entropy_list
    })

In [22]:
results.head()

,name_1,R,G,B,H,S,V,lbp_energy,lbp_entropy,contrast_new,dissimilarity_new,homogeneity_new,energy_new,correlation_new,gabor_energy,gabor_entropy,cognitive_demand,shanon_entropy
0,I000683589.jpg,57.291932,67.955537,81.675283,106.051279,94.914521,81.684192,0.142754,2.904530,1308.058197,23.115830,0.099700,0.015093,0.760651,0.449064,1.818206,74.888132,7.262302
1,I000241412.jpg,161.064373,99.694436,64.482302,14.182597,171.450813,161.219846,0.159242,2.813289,3129.569096,37.796614,0.053688,0.008898,0.619590,0.228516,2.541085,80.981043,7.722289
2,I000597539.jpg,78.793945,85.626674,51.470544,42.092714,120.281948,93.879006,0.145569,2.881483,1086.762991,21.020512,0.103080,0.017609,0.845360,0.332351,2.151809,73.284956,7.362419
3,I000399706.jpg,167.279616,150.449617,134.850566,29.697026,77.731405,169.987962,0.168028,2.769595,2210.527087,25.655385,0.185728,0.082968,0.821773,0.203346,2.659158,70.272792,7.422151
4,I000308594.jpg,180.949498,180.949498,180.949498,0.000000,0.000000,180.949498,0.147855,2.865537,652.321167,12.766389,0.191146,0.024382,0.925043,0.145982,2.891916,67.860898,7.500473


In [21]:
results.drop(['name_2'],axis=1,inplace=True)

In [32]:
results.rename(columns = {'name_1':'iid'}, inplace = True)

In [33]:
results.shape

(39278, 18)

In [42]:
results.head()

,iid,R,G,B,H,S,V,lbp_energy,lbp_entropy,contrast_new,dissimilarity_new,homogeneity_new,energy_new,correlation_new,gabor_energy,gabor_entropy,cognitive_demand,shanon_entropy
0,I000683589.jpg,57.291932,67.955537,81.675283,106.051279,94.914521,81.684192,0.142754,2.904530,1308.058197,23.115830,0.099700,0.015093,0.760651,0.449064,1.818206,74.888132,7.262302
1,I000241412.jpg,161.064373,99.694436,64.482302,14.182597,171.450813,161.219846,0.159242,2.813289,3129.569096,37.796614,0.053688,0.008898,0.619590,0.228516,2.541085,80.981043,7.722289
2,I000597539.jpg,78.793945,85.626674,51.470544,42.092714,120.281948,93.879006,0.145569,2.881483,1086.762991,21.020512,0.103080,0.017609,0.845360,0.332351,2.151809,73.284956,7.362419
3,I000399706.jpg,167.279616,150.449617,134.850566,29.697026,77.731405,169.987962,0.168028,2.769595,2210.527087,25.655385,0.185728,0.082968,0.821773,0.203346,2.659158,70.272792,7.422151
4,I000308594.jpg,180.949498,180.949498,180.949498,0.000000,0.000000,180.949498,0.147855,2.865537,652.321167,12.766389,0.191146,0.024382,0.925043,0.145982,2.891916,67.860898,7.500473


In [41]:
results.to_csv("df_iesn_results_low_level_features.csv")